In [ ]:
import time
import cv2
import numpy as np
from rembg import new_session, remove
from PIL import Image

def benchmark_model_on_camera(model_name: str, runs: int = 100, cam_index: int = 0):
    # Initialize camera
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open camera index {cam_index}")
    
    # Warm up camera
    for _ in range(10):
        cap.read()
    
    # Create a persistent rembg session
    session = new_session(model_name)
    
    times = []
    for i in range(runs):
        ret, frame = cap.read()
        if not ret:
            raise RuntimeError("Camera frame grab failed")
        
        # Convert to PIL and time only the remove() call + materialization
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(rgb)
        
        start = time.perf_counter()
        out = remove(pil, session=session)
        _ = np.array(out)    # force decode
        elapsed = time.perf_counter() - start
        
        times.append(elapsed)
    
    cap.release()
    avg_ms = (sum(times) / len(times)) * 1000
    print(f"[{model_name:8s}] average: {avg_ms:.2f} ms over {runs} frames")

if __name__ == "__main__":
    RUNS = 1
    models = (
        "u2net",                 # full-size general
        "silueta",               # pruned U²-Net
        "isnet-general-use",     # ISNet general
        "birefnet-general",      # BiRefNet general
        "birefnet-general-lite", # lighter BiRefNet
        "u2netp",                # pico U²-Net (your single ultra-light model)
    )
    for model in models:
        benchmark_model_on_camera(model, runs=RUNS)
